In [ ]:
import google.generativeai as genai
import os
import json
import re
import magic
from PIL import Image
import io
from pillow_heif import register_heif_opener
register_heif_opener()

genai.configure(api_key="Enter your Google gemini API")

def get_gemini_response(input_text, image, prompt):
    model = genai.GenerativeModel('gemini-1.5-flash')
    response = model.generate_content([input_text, image[0], prompt])
    return response.text

def input_image_setup(uploaded_file):
    if uploaded_file is not None:
        bytes_data = uploaded_file.read()

        mime_type = magic.from_buffer(bytes_data, mime=True)

        image_parts = [
            {
                "mime_type": mime_type,  # Use to detected MIME type
                "data": bytes_data
            }
        ]
        return image_parts
    else:
        raise FileNotFoundError("No file uploaded")

def input_image_setup(uploaded_file):
    if uploaded_file is not None:
        bytes_data = uploaded_file.read()

        mime_type = magic.from_buffer(bytes_data, mime=True)
        image = Image.open(io.BytesIO(bytes_data))

        if mime_type == 'image/heic':
            image = image.convert("RGB")
            output = io.BytesIO()
            image.save(output, format="JPEG")
            image_data = output.getvalue()
            mime_type = 'image/jpeg'
        else:
            image_data = bytes_data

        image_parts = [
            {
                "mime_type": mime_type,
                "data": image_data
            }
        ]
        return image_parts
    else:
        raise FileNotFoundError("No file uploaded")

def main(json_list):
#you can give any input
    input_text = ("You are an AI bot designed to act as a professional for parsing resumes you give everything in a structured format. You are given a resume, and your job is to extract the following information:( 1. Full name 2. Email ID 3. Phone number 4. College details 5. Technical skil 6. Certifications  7. Project(name only and tools and technologies used in that particular project)) and return the response in json format and Keys should be in uppar case, format of keys should remain same throughout")

    image_list = os.listdir("/content")
    for image in image_list:
          if image.endswith((".jpg", ".jpeg", ".png", ".heic")):
            print(image)
            image_path = os.path.join("/content/", image)
            try:
              with open(image_path, "rb") as uploaded_file:

                  image_data = input_image_setup(uploaded_file)

                #can give any input in prompt or with use case
                  input_prompt = """
                      You are an AI bot designed to act as a professional for parsing resumes and you give everything in a structured format, You give result of parsed resumes in same format for all resumes. You are given a resume, and your job is to extract the following information: 1. Full name 2. Email ID 3. Phone number 4. College details 5. Technical skil 6. Certifications  7. Project(name only and tools and technologies used in that particular project) and return the response in json format and Keys should be in uppar case and format of keys should remain same throught.

                  """
                  response_text = get_gemini_response(input_text, image_data, input_prompt)

                  print("\nThe Raw Response is:")
                  print(response_text)

                  cleaned_response = response_text.replace("json","").replace("```","")

                  print("\nThe Parsed Result is:")
                  print(json.loads(cleaned_response))
                  json_list.append(json.loads(cleaned_response))

            except Exception as e:
              print(f"Error: {e}")

if __name__ == "__main__":
    json_list = []
    main(json_list)
with open('result2.json','w') as json_file:
  json.dump(json_list,json_file)

In [ ]:
import pandas as pd
df = pd.read_json('/content/result2.json')
df.to_csv('Test2.csv', index=False)


In [ ]:
!pip install google-generativeai
!pip install pillow-heif
!pip install pillow
!pip install python-magic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 67.6 MB/s eta 0:00:00
